In [2]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8002/v1", api_key="not-needed")

resp = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B-Instruct-AWQ",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain retrieval-augmented generation briefly."}
    ]
)

print(resp.choices[0].message.content)


InternalServerError: Error code: 503

In [3]:
import requests
import json

q = "describe climate change in 2023"

url = "http://localhost:8002/v1/chat/completions"

payload = {
    # "model": "Qwen/Qwen2.5-7B-Instruct-AWQ",
    "model": "Qwen/Qwen3-4B-AWQ",
    "messages": [
        {"role": "user", "content": q}
    ],
    "stream": True
}

headers = {
    "Content-Type": "application/json"
}

with requests.post(url, json=payload, headers=headers, stream=True) as r:
    for line in r.iter_lines():
        if line:
            decoded = line.decode("utf-8")
            if decoded.startswith("data: "):
                data = decoded[len("data: "):]
                if data.strip() == "[DONE]":
                    break
                try:
                    content = json.loads(data)["choices"][0]["delta"].get("content", "")
                    print(content, end="", flush=True)
                except Exception as e:
                    pass

print("\n--- Stream finished ---")


<think>
Okay, I need to describe climate change in 2023. Let me start by recalling what I know about recent climate data. First, 2023 was a significant year for climate events. I remember that 2023 was one of the warmest years on record. The Earth's temperature was probably higher than previous years. I should check if that's accurate. Also, there were some major weather events in 2023, like extreme heatwaves, wildfires, and maybe some hurricanes or typhoons. 

I should mention the role of greenhouse gases. The concentration of CO2 in the atmosphere has been rising, and human activities are the main cause. Maybe I should include some statistics, like the CO2 levels in 2023. Also, the Intergovernmental Panel on Climate Change (IPCC) reports are important here. They might have released a report in 2023 that highlights the urgency of the situation. 

There's also the aspect of climate change impacts on different regions. For example, some areas might have experienced more intense rainfall

Test keyword extractor

In [ ]:
from openai import OpenAI
import requests
import json

In [1]:
keyword_extractor_prompt = """You are a smart AI assistant helping with document retrieval.
Your job is to extract ONLY the essential content-bearing keywords and phrases from a user query.
These keywords must be useful for vector database search.

Important rules:
- Include only nouns, proper nouns, entities, domain-specific terms, and technical concepts.
- Exclude generic verbs (e.g., explain, analyze, describe), adjectives, adverbs, stopwords, and pronouns.
- Do NOT include words like: what, how, effects, explain, impact, analysis, describe, definition, etc.
- Focus on core topics, entities, and technical terms.
- Return a short, minimal list of distinct terms or phrases.
- Output must be strictly a comma-separated list, and nothing else.

Examples:

User: What are the effects of climate change on polar bear populations? explain briefly.
Keywords: climate change, polar bears

User: How does the Transformer architecture work in deep learning?
Keywords: Transformer, deep learning

User: Explain the effects of railroads in Britain.
Keywords: railroads, Britain

User: سلام، بگو ببینم حافظه کاری در روانشناسی چی هست؟
Keywords: حافظه کاری, روانشناسی

Now extract keywords from the following query:
"""

In [11]:
text = "what country started industrial revolution in the world?"

client = OpenAI(base_url="http://localhost:8002/v1", api_key="not-needed")
prompt = f"{keyword_extractor_prompt}User: {text}\nKeywords:"

resp = client.chat.completions.create(
    model="Qwen/Qwen3-4B-AWQ",
    messages=[
        {"role": "system", "content": "You are a keyword extraction assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=30,
    extra_body={  # 👇 disables thinking traces
        "chat_template_kwargs": {"enable_thinking": False}
    }
)

generated_text = resp.choices[0].message.content.strip()

if "Keywords:" in generated_text:
    extracted_keywords = generated_text.split("Keywords:")[-1].strip()
else:
    extracted_keywords = generated_text

print("Extracted Keywords:", extracted_keywords)

Extracted Keywords: industrial revolution, country
